# Regression Analysis: Factors Influencing Loan Amount

## Data Cleaning and Preprocessing

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)
sns.set(style='whitegrid', palette='pastel')

# Load dataset
df = pd.read_csv('DS_assignment_data.csv', thousands=',', encoding='latin1')
print(f"Dataset shape: {df.shape}")

Dataset shape: (233154, 37)


### Initial Data Exploration


In [6]:
# Display initial information
print("\nData Types and Missing Values:")
print(df.info())

# Summary statistics
print("\nSummary Statistics:")
print(df.describe().transpose())

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum().sort_values(ascending=False))


Data Types and Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 37 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UNIQUEID                             233154 non-null  int64  
 1   DISBURSED_AMOUNT                     233154 non-null  int64  
 2   ASSET_COST                           233154 non-null  int64  
 3   LTV                                  233154 non-null  float64
 4   BRANCH_ID                            233154 non-null  int64  
 5   CURRENT_PINCODE_ID                   233154 non-null  int64  
 6   DATE_OF_BIRTH                        233154 non-null  object 
 7   EMPLOYMENT_TYPE                      225493 non-null  object 
 8   DISBURSAL_DATE                       233154 non-null  object 
 9   MOBILENO_AVL_FLAG                    233154 non-null  int64  
 10  AADHAR_FLAG                          233154 non-

### Data Cleaning


In [7]:
# ----------------------------
# Data Cleaning and Preprocessing
# ----------------------------

def clean_currency_columns(df):
    """Clean columns with currency symbols and commas"""
    for col in ['DISBURSED_AMOUNT', 'ASSET_COST']:
        if df[col].dtype == 'object':
            df[col] = df[col].replace('€ ', '', regex=True).str.replace(',', '').astype(float)
    return df

def convert_to_months(s):
    """Convert average account age from string to months"""
    if pd.isna(s):
        return np.nan
    parts = s.split()
    years = months = 0
    for part in parts:
        if 'yrs' in part:
            years = int(part.replace('yrs', ''))
        elif 'mon' in part:
            months = int(part.replace('mon', ''))
    return years * 12 + months

def preprocess_data(df):
    """Main data preprocessing function"""
    # Clean currency columns
    df = clean_currency_columns(df)
    
    # Convert date columns
    df['DISBURSAL_DATE'] = pd.to_datetime(df['DISBURSAL_DATE'], dayfirst=True)
    df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], dayfirst=True, errors='coerce')
    
    # Calculate age
    df['AGE'] = (df['DISBURSAL_DATE'] - df['DATE_OF_BIRTH']).dt.days // 365
    
    # Convert average account age to months
    df['AVERAGE_ACCT_AGE_MONTHS'] = df['AVERAGE_ACCT_AGE'].apply(convert_to_months)
    
    # Handle employment type
    df['EMPLOYMENT_TYPE'] = df['EMPLOYMENT_TYPE'].fillna(df['EMPLOYMENT_TYPE'].mode()[0])
    le = LabelEncoder()
    df['EMPLOYMENT_TYPE_ENCODED'] = le.fit_transform(df['EMPLOYMENT_TYPE'])
    
    # Clean credit score
    df['PERFORM_CNS_SCORE'] = pd.to_numeric(df['PERFORM_CNS_SCORE'], errors='coerce')
    
    # Drop unnecessary columns
    df = df.drop(columns=['DATE_OF_BIRTH', 'DISBURSAL_DATE', 'AVERAGE_ACCT_AGE', 
                         'CREDIT_HISTORY_LENGTH', 'PERFORM_CNS_SCORE_DESCRIPTION'])
    
    return df

# Apply preprocessing
df_clean = preprocess_data(df)
print("\nAfter cleaning:")
print(f"Dataset shape: {df_clean.shape}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")

# Create employment type flags
df_clean['EMPLOYMENT_TYPE_Self_Employed'] = (df_clean['EMPLOYMENT_TYPE'] == 'Self employed').astype(int)
df_clean['EMPLOYMENT_TYPE_Salaried'] = (df_clean['EMPLOYMENT_TYPE'] == 'Salaried').astype(int)

# Final modeling dataset
features = [
    'ASSET_COST', 'LTV', 'EMPLOYMENT_TYPE_Self_Employed',
    'PERFORM_CNS_SCORE', 'PRI_NO_OF_ACCTS', 'PRI_CURRENT_BALANCE',
    'PRIMARY_INSTAL_AMT', 'NEW_ACCTS_IN_LAST_SIX_MONTHS',
    'AVERAGE_ACCT_AGE_MONTHS', 'NO_OF_INQUIRIES'
]
target = 'DISBURSED_AMOUNT'
df_model = df_clean[features + [target]].dropna()
print(f"\nFinal modeling dataset shape: {df_model.shape}")

C:\Users\jeffr\AppData\Local\Temp\ipykernel_44880\2946351842.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DISBURSAL_DATE'] = pd.to_datetime(df['DISBURSAL_DATE'], dayfirst=True)
C:\Users\jeffr\AppData\Local\Temp\ipykernel_44880\2946351842.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], dayfirst=True, errors='coerce')



After cleaning:
Dataset shape: (233154, 35)
Missing values: 0

Final modeling dataset shape: (233154, 11)


###  Final Dataset for Modeling



In [8]:
# Create employment type flags
df_clean['EMPLOYMENT_TYPE_Self_Employed'] = (df_clean['EMPLOYMENT_TYPE'] == 'Self employed').astype(int)
df_clean['EMPLOYMENT_TYPE_Salaried'] = (df_clean['EMPLOYMENT_TYPE'] == 'Salaried').astype(int)

# Final modeling dataset
features = [
    'ASSET_COST', 'LTV', 'EMPLOYMENT_TYPE_Self_Employed',
    'PERFORM_CNS_SCORE', 'PRI_NO_OF_ACCTS', 'PRI_CURRENT_BALANCE',
    'PRIMARY_INSTAL_AMT', 'NEW_ACCTS_IN_LAST_SIX_MONTHS',
    'AVERAGE_ACCT_AGE_MONTHS', 'NO_OF_INQUIRIES'
]
target = 'DISBURSED_AMOUNT'
df_model = df_clean[features + [target]].dropna()
print(f"\nFinal modeling dataset shape: {df_model.shape}")


Final modeling dataset shape: (233154, 11)
